<a href="https://colab.research.google.com/github/saurabhkirar/NLP_END/blob/main/Session8/Session8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Write a pyth0n program to multiply two number 
num1=100.19
num2=200.43
num3=num1*num2
print('The product of %.2f and %.2f is %.2f'%(num1,num2,num3))

The product of 100.19 and 200.43 is 20081.08


In [ ]:
# Write a program to print the positive number of all the  given number in a list 
lis=[1,3,-9,-81,7776,-9876]
abs_list=[]
print('actual list',lis)
for rec in lis:
  rec=abs(rec)
  abs_list.append(rec)
print('list with absolute values',abs_list)

actual list [1, 3, -9, -81, 7776, -9876]
list with absolute values [1, 3, 9, 81, 7776, 9876]


In [ ]:
## Write a program that has a python function to return the value if the input  is greater than zero else the function should return zero
# test the function with one postive and one negetive value.
def func_relu(x):
  return x if x > 0 else 0

print('the output for 98 is ',func_relu(98))
print('the output for -98  is ',func_relu(-98))

the output for 98 is  98
the output for -98  is  0


In [ ]:
## write a python code to find the maximum and mimimum values in a list.
lst=[98,776,1,22,33]
print('Maximum value in the list is ',max(lst))
print('minimum vale in the list is ',min(lst))

Maximum value in the list is  776
minimum vale in the list is  1


In [ ]:
## Write a function in python which takes a list as an input and returns the normalized list as the output.
## x normalized = (x – x minimum) / (x maximum – x minimum)
## test the function and print the normalized list
def func_normalization(p_list):
  l_max=max(p_list)
  l_min=min(p_list)
  deno=l_max-l_min
  norm_lst=[]
  for rec in p_list:
    norm_lst.append((rec-l_min)/deno)
  return norm_lst

nat_list=[234,345,887,98]
func_normalization(nat_list)

[0.17237008871989862, 0.31305449936628643, 1.0, 0.0]

In [ ]:
## Write a code to print the  memory address of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('Memory address of the given array is ',arr.buffer_info())

Memory address of the given array is  (140549508420864, 4)


In [ ]:
## Write a code to print the  no of element of a given array 
from array import *
arr=array('i',[1,5,7,89])
print('There are total %d elements '%(arr.buffer_info()[1]))

There are total 4 elements 


In [ ]:
#Write a Python program to remove the first occurrence of a specified element from an array
from array import *
array_num = array('i', [1, 3, 15, 3, 37, 21, 29, 3])
print("Original array: "+str(array_num))
array_num.remove(3)
print("New array: "+str(array_num))

Original array: array('i', [1, 3, 15, 3, 37, 21, 29, 3])
New array: array('i', [1, 15, 3, 37, 21, 29, 3])


In [ ]:
## write a code to define dictionary and loop over it.
d = {'x': 10, 'y': 20, 'z': 30} 
for dict_key, dict_value in d.items():
    print(dict_key,'->',dict_value)

x -> 10
y -> 20
z -> 30


In [ ]:
## Write a code to merge two dictionaries.
d1 = {'a': 100, 'b': 200}
d2 = {'x': 300, 'y': 200}
d = d1.copy()
d.update(d2)
print(d)

{'a': 100, 'b': 200, 'x': 300, 'y': 200}


In [ ]:
## write a code to merge two lists into one dictionary.
keys = ['sachin', 'hayden', 'sana']
values = ['Ind','Aus', 'SL']
player = dict(zip(keys, values))
print(player)

{'sachin': 'Ind', 'hayden': 'Aus', 'sana': 'SL'}


In [ ]:
## write a class in oythin to get the dictionary from an object field, test the code.
class Player(object):
     def __init__(self):
         self.ind = 'sachin'
         self.aus = 'hayden'
         self.sl = 'sana'
     def do_nothing(self):
         pass
test = Player()
print(test.__dict__)

{'ind': 'sachin', 'aus': 'hayden', 'sl': 'sana'}


In [ ]:
"""
Write a Python program to print all unique values in a dictionary. Go to the editor
Sample Data : [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
Expected Output : Unique Values: {'5', '2', '7', '1', '9'}
"""
L = [{"V":"1"}, {"V": "2"}, {"VI": "1"}, {"VI": "5"}, {"VII":"5"}, {"V":"9"},{"VIII":"7"}]
print("Original List: ",L)
u_value = set( val for dic in L for val in dic.values())
print("Unique Values: ",u_value)

Original List:  [{'V': '1'}, {'V': '2'}, {'VI': '1'}, {'VI': '5'}, {'VII': '5'}, {'V': '9'}, {'VIII': '7'}]
Unique Values:  {'2', '1', '7', '9', '5'}


In [ ]:
## Write a pythin code to add three lists using lambda and map.
nums1 = [1, 2, 3] 
nums2 = [4, 5, 6] 
nums3 = [7, 8, 9] 
print("Original list: ")
print(nums1)  
print(nums2)  
print(nums3)  
result = map(lambda x, y, z: x + y + z, nums1, nums2, nums3) 
print("\nNew list after adding above three lists:")
print(list(result))

Original list: 
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

New list after adding above three lists:
[12, 15, 18]


In [ ]:
## use time and datetine and list down few formats 
import time
import datetime
print("Current date and time: " , datetime.datetime.now())
print("Current year: ", datetime.date.today().strftime("%Y"))
print("Month of year: ", datetime.date.today().strftime("%B"))
print("Week number of the year: ", datetime.date.today().strftime("%W"))
print("Weekday of the week: ", datetime.date.today().strftime("%w"))
print("Day of year: ", datetime.date.today().strftime("%j"))
print("Day of the month : ", datetime.date.today().strftime("%d"))
print("Day of week: ", datetime.date.today().strftime("%A"))

Current date and time:  2020-12-16 15:35:07.374382
Current year:  2020
Month of year:  December
Week number of the year:  50
Weekday of the week:  3
Day of year:  351
Day of the month :  16
Day of week:  Wednesday


In [ ]:
## Write a code to show the unique file type based on the extension from a list.

lst=['a.jpg','b.png','c.jpeg','d.java','e.sql','f.sql','g.sql','z.png']
uniq=[]
for rec in lst:
    E =rec.split('.')[1]
    uniq.append(E)
set(uniq)  ## Set gives the distinct or non repetative values of the list.

{'java', 'jpeg', 'jpg', 'png', 'sql'}

In [ ]:
## Write a python function to return the greatest common divisor (GCD) for two numbers, test the functions.
def gcd(x, y):
    gcd = 1
    
    if x % y == 0:
        return y
    
    for k in range(int(y / 2), 0, -1):
        if x % k == 0 and y % k == 0:
            gcd = k
            break  
    return gcd

print(gcd(12, 17))
print(gcd(4, 6))

1
2


In [ ]:
## Write a code to calculate the euclidiean distance between two cordinates, express the cordinates  as list members.
import math
p1 = [4, 0]
p2 = [6, 6]
distance = math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )

print(distance)

6.324555320336759


In [ ]:
## Write a code to see if any file exists in the location.
import os.path
open('abc.txt', 'w')
print(os.path.isfile('abc.txt'))

True


In [ ]:
## Write a code to find out number of CPU using.
import multiprocessing
print(multiprocessing.cpu_count())

2


In [ ]:
## Wite a code to find out profiling of python programs.
import cProfile
def res():
    print(100.72*344.87)
cProfile.run('res()')

34735.3064
         39 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-17-a8dede55eddf>:3(res)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:382(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:432(send)
        3    0.000    0.000    0.000    0.000 threading.py:1062(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
        3    0.000    0.000    0.000    0.000 threading.py:506(is_set)
        1    0.0

In [ ]:
## Write a program to get the current user name 
import getpass
print(getpass.getuser())

root


In [ ]:
## Write a code to print the URL content to the console.
from http.client import HTTPConnection
conn = HTTPConnection("google.com")
conn.request("GET", "/")  
result = conn.getresponse()
# retrieves the entire contents.  
contents = result.read() 
print(contents)

b'<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">\n<TITLE>301 Moved</TITLE></HEAD><BODY>\n<H1>301 Moved</H1>\nThe document has moved\n<A HREF="http://www.google.com/">here</A>.\r\n</BODY></HTML>\r\n'


In [ ]:
## write a pyhon function to give the running total and test with with a list.
from itertools import accumulate
import operator
def running_sum(it):
    return accumulate(it,operator.add)

#List
result = running_sum([1,2,3,4,5,6,7])
print("Running sum of a list:")
for i in result:
    print(i)

Running sum of a list:
1
3
6
10
15
21
28


In [ ]:
#Write a Python program to check that a string contains only a certain set of characters (in this case a-z, A-Z and 0-9).
import re
def is_allowed_specific_char(string):
    charRe = re.compile(r'[^a-zA-Z0-9.]')
    string = charRe.search(string)
    return not bool(string)

print(is_allowed_specific_char("ABCDEFabcdef123450")) 
print(is_allowed_specific_char("*&%@#!}{"))

True
False


In [ ]:
#Write a Python program that iterate over elements repeating each as many times as its count.
from collections import Counter
c = Counter(p=4, q=2, r=0, s=-2)
print(list(c.elements()))


['p', 'p', 'p', 'p', 'q', 'q']


In [ ]:
## find the most common words in a sentence.
from collections import Counter
import re
text = """The start of T20 in a way revolutionised the cricket industry for good.  When we say this it is not just in terms of sports, fame, and viewership but also how it pumped up cricketer’s income. There are various leagues like BBL, IPL, CPL, PSL, and BPL which are pulling several big names across the globe which automatically helps in garnering massive revenue.

With the genesis of T20 cricket, cricket has witnessed revolutionary changes in terms of the popularity of the sport and the income earned by cricketers. The format has brought in a fast-moving, entertaining form of cricket to the world, forcing batsmen and bowlers to garner out-of-the-box solutions to counter the attacking prowess of the opponent.."""
words = re.findall('\w+',text)
print(Counter(words).most_common(10))

[('the', 10), ('of', 8), ('in', 5), ('cricket', 4), ('and', 4), ('to', 3), ('The', 2), ('T20', 2), ('a', 2), ('it', 2)]


In [ ]:
#Write a Python program to find the most common elements and their counts of a specified text.
from collections import Counter
s = 'lkseropewdssafsdfafkpwe'
print("Original string: "+s)
print("Most common three characters of the said string:")
print(Counter(s).most_common(3))

Original string: lkseropewdssafsdfafkpwe
Most common three characters of the said string:
[('s', 4), ('e', 3), ('f', 3)]


In [ ]:
## Write a code to merge two dictionaries 
import collections as ct
def merge_dictionaries(color1,color2):
    merged_dict = dict(ct.ChainMap({}, color1, color2))
    return merged_dict
color1 = { "R": "Red", "B": "Black", "P": "Pink" }
color2 = { "G": "Green", "W": "White" }
print("Original dictionaries:")
print(color1,' ',color2)
print("\nMerged dictionary:")
print(merge_dictionaries(color1, color2))

Original dictionaries:
{'R': 'Red', 'B': 'Black', 'P': 'Pink'}   {'G': 'Green', 'W': 'White'}

Merged dictionary:
{'W': 'White', 'G': 'Green', 'R': 'Red', 'P': 'Pink', 'B': 'Black'}


In [ ]:
## write a code to print count of each element in the list.
sampleList = [11, 45, 8, 11, 23, 45, 23, 45, 89]
print("Original list ", sampleList)

countDict = dict()
for item in sampleList:
  if(item in countDict):
    countDict[item] += 1
  else:
    countDict[item] = 1
  
print("Printing count of each item  ",countDict)

Original list  [11, 45, 8, 11, 23, 45, 23, 45, 89]
Printing count of each item   {11: 2, 45: 3, 8: 1, 23: 2, 89: 1}


In [ ]:
##Given a following two sets find the intersection and remove those elements from the first set
firstSet  = {23, 42, 65, 57, 78, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}
print("First Set ", firstSet)
print("Second Set ", secondSet)

intersection = firstSet.intersection(secondSet)
print("Intersection is ", intersection)
for item in intersection:
  firstSet.remove(item)

print("First Set after removing common element ", firstSet)

First Set  {65, 42, 78, 83, 23, 57, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
Intersection is  {57, 83, 29}
First Set after removing common element  {65, 42, 78, 23}


In [ ]:
##Given two sets, Checks if One Set is Subset or superset of Another Set. if the subset is found delete all elements from that set
firstSet  = {57, 83, 29}
secondSet = {57, 83, 29, 67, 73, 43, 48}

print("First Set ", firstSet)
print("Second Set ", secondSet)

print("First set is subset of second set -", firstSet.issubset(secondSet))
print("Second set is subset of First set - ", secondSet.issubset(firstSet))

print("First set is Super set of second set - ", firstSet.issuperset(secondSet))
print("Second set is Super set of First set - ", secondSet.issuperset(firstSet))

if(firstSet.issubset(secondSet)):
  firstSet.clear()
  
if(secondSet.issubset(firstSet)):
  secondSet.clear()
print("First Set ", firstSet)
print("Second Set ", secondSet)

First Set  {57, 83, 29}
Second Set  {67, 73, 43, 48, 83, 57, 29}
First set is subset of second set - True
Second set is subset of First set -  False
First set is Super set of second set -  False
Second set is Super set of First set -  True
First Set  set()
Second Set  {67, 73, 43, 48, 83, 57, 29}


In [ ]:
"""
Write a program that accepts a comma separated sequence of words as input and prints the words in a comma-separated sequence after sorting them alphabetically.
Suppose the following input is supplied to the program:
without,hello,bag,world
Then, the output should be:
bag,hello,without,world
"""
r_input='without,hello,bag,world'

items=[x for x in r_input.split(',')]
items.sort()
print (','.join(items))

bag,hello,without,world


In [ ]:
##Write a program that accepts a sequence of whitespace separated words as input and prints the words after removing all duplicate words and sorting them alphanumerically.
##Suppose the following input is supplied to the program:
##hello world and practice makes perfect and hello world again
##Then, the output should be:
##again and hello makes perfect practice world

s = 'hello world and practice makes perfect and hello world again'
words = [word for word in s.split(" ")]
print(" ".join(sorted(list(set(words)))))

again and hello makes perfect practice world


In [ ]:
#Write a program that accepts a sentence and calculate the number of letters and digits.
#Suppose the following input is supplied to the program:
#hello world! 123
#Then, the output should be:
##LETTERS 10
#DIGITS 3

s = 'mai to raste se ja raha tha #nh247'
d={"DIGITS":0, "LETTERS":0}
for c in s:
    if c.isdigit():
        d["DIGITS"]+=1
    elif c.isalpha():
        d["LETTERS"]+=1
    else:
        pass
print ("LETTERS", d["LETTERS"])
print ("DIGITS", d["DIGITS"])

LETTERS 23
DIGITS 3


In [ ]:
#write a code which uses a list comprehension to square each odd number in a list. The list is input by a sequence of comma-separated numbers.

values = '1,2,3,4,5,6,7,8,9'
numbers = [x for x in values.split(",") if int(x)%2!=0]
print(",".join(numbers))

1,3,5,7,9


In [ ]:
#Write a program to compute the frequency of the words from the input. The output should output after sorting the key alphanumerically. 

raw_input='New to Python or choosing between Python 2 and Python 3? Read Python 2 or Python 3.'

freq = {}   # frequency of words in text
line = raw_input
for word in line.split():
    freq[word] = freq.get(word,0)+1

words = freq.keys()

for w in words:
    print ("%s:%d" % (w,freq[w]))

New:1
to:1
Python:5
or:2
choosing:1
between:1
2:2
and:1
3?:1
Read:1
3.:1


In [ ]:

#Define a function that can accept two strings as input and print the string with maximum length in console. 
#If two strings have the same length, then the function should print al l strings line by line.
def printValue(s1,s2):
	len1 = len(s1)
	len2 = len(s2)
	if len1>len2:
		print (s1)
	elif len2>len1:
		print (s2)
	else:
		print (s1)
		print (s2)
		
printValue("one","three")

three


In [ ]:
#Define a function which can generate a dictionary where the keys are numbers between 1 and 20 (both included) and the values are square of keys. The function should just print the values only.

def printDict():
	d=dict()
	for i in range(1,21):
		d[i]=i**2
	for (k,v) in d.items():	
		print( v)
		

printDict()

1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400


In [ ]:

#Write a program which can filter even numbers in a list by using filter function. The list is: [1,2,3,4,5,6,7,8,9,10].

li = [1,2,3,4,5,6,7,8,9,10]
evenNumbers = filter(lambda x: x%2==0, li)
print(evenNumbers)


In [ ]:
#Define a class named Circle which can be constructed by a radius. The Circle class has a method which can compute the area. 

class Circle(object):
    def __init__(self, r):
        self.radius = r

    def area(self):
        return self.radius**2*3.14

aCircle = Circle(2)
print (aCircle.area())


12.56


In [ ]:
#Define a class named Rectangle which can be constructed by a length and width. The Rectangle class has a method which can compute the area. 

class Rectangle(object):
    def __init__(self, l, w):
        self.length = l
        self.width  = w

    def area(self):
        return self.length*self.width

aRectangle = Rectangle(2,10)
print(aRectangle.area())

20


In [ ]:
#Write a function to compute 5/0 and use try/except to catch the exceptions.

def throws():
    return 5/0

try:
    throws()
except ZeroDivisionError:
    print ("division by zero!")
except Exception:
    print ('Caught an exception')
finally:
    print ('In finally block for cleanup')


division by zero!
In finally block for cleanup


In [ ]:
#Assuming that we have some email addresses in the "username@companyname.com" format, please write program to print the company name of a given email address. Both user names and company names are composed of letters only.


raw_input='user@company.com'

import re
emailAddress = raw_input
pat2 = "(\w+)@(\w+)\.(com)"
r2 = re.match(pat2,emailAddress)
print (r2.group(2))

company


In [ ]:
#Write a program which accepts a sequence of words separated by whitespace as input to print the words composed of digits only.

raw_input='7 bad habbits of 9 ineffective people, 1 should be procrastination'
import re
s = raw_input
print(re.findall("\d+",s))

['7', '9', '1']


In [ ]:
#Please write a binary search function which searches an item in a sorted list. The function should return the index of element to be searched in the list.

import math
def bin_search(li, element):
    bottom = 0
    top = len(li)-1
    index = -1
    while top>=bottom and index==-1:
        mid = int(math.floor((top+bottom)/2.0))
        if li[mid]==element:
            index = mid
        elif li[mid]>element:
            top = mid-1
        else:
            bottom = mid+1

    return index

li=[2,5,7,9,11,17,222]
print (bin_search(li,11))
print (bin_search(li,12))

4
-1


In [ ]:
#Please generate a random float where the value is between 10 and 100 using Python math module.
import random
print (random.random()*100)

43.452726663473065


In [ ]:
#Please write a program to compress and decompress the string "hello world!hello world!hello world!hello world!".
import zlib
s = "hello world!hello world!hello world!hello world!"
t = zlib.compress(s)
print (t)
print (zlib.decompress(t))

print (random.choice([i for i in range(11) if i%2==0])

In [ ]:
# write a program which accepts a string from console and print it in reverse order.
raw_input='my name is, well whats there in the name '

s=raw_input
s = s[::-1]
print (s)

 eman eht ni ereht stahw llew ,si eman ym


In [ ]:
#write a program which prints all permutations of [1,2,3]

import itertools
print (list(itertools.permutations([1,2,3])))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


In [ ]:
#Write a funtion  which can compute the factorial of a given numbers.

def fact(x):
    if x == 0:
        return 1
    return x * fact(x - 1)

x=int(9)
fact(x)

362880

In [ ]:
#Write a program that accepts a sentence and calculate the number of upper case letters and lower case letters.

s = 'This Is tHe SEntencE'
d={"UPPER CASE":0, "LOWER CASE":0}
for c in s:
    if c.isupper():
        d["UPPER CASE"]+=1
    elif c.islower():
        d["LOWER CASE"]+=1
    else:
        pass
print ("UPPER CASE", d["UPPER CASE"])
print ("LOWER CASE", d["LOWER CASE"])

UPPER CASE 6
LOWER CASE 11


In [ ]:
"""
A website requires the users to input username and password to register. Write a program to check the validity of password input by users.
Following are the criteria for checking the password:
1. At least 1 letter between [a-z]
2. At least 1 number between [0-9]
1. At least 1 letter between [A-Z]
3. At least 1 character from [$#@]
4. Minimum length of transaction password: 6
5. Maximum length of transaction password: 12
Your program should accept a sequence of comma separated passwords and will check them according to the above criteria. Passwords that match the criteria are to be printed, each separated by a comma.
Example
If the following passwords are given as input to the program:
ABd1234@1,a F1#,2w3E*,2We3345
Then, the output of the program should be:
ABd1234@1
"""
r_input='Wak3n@9ends'
import re
value = []
items=[x for x in r_input.split(',')]
for p in items:
    if len(p)<6 or len(p)>12:
        continue
    else:
        pass
    if not re.search("[a-z]",p):
        continue
    elif not re.search("[0-9]",p):
        continue
    elif not re.search("[A-Z]",p):
        continue
    elif not re.search("[$#@]",p):
        continue
    elif re.search("\s",p):
        continue
    else:
        pass
    value.append(p)
print (",".join(value))
#---------------

Wak3n@9ends


In [ ]:
#Define a class Person and its two child classes: Male and Female. All classes have a method "getGender" which can print "Male" for Male class and "Female" for Female class.

class Person(object):
    def getGender(self):
        return "Unknown"


class Male(Person):
    def getGender(self):
        return "Male"


class Female(Person):
    def getGender(self):
        return "Female"


aMale = Male()
aFemale = Female()
print(aMale.getGender())
print(aFemale.getGender())

Male
Female


In [ ]:
## Write a program to find the resolution of an image.
def jpeg_res(filename):
   """"This function prints the resolution of the jpeg image file passed into it"""

   # open image for reading in binary mode
   with open(filename,'rb') as img_file:

       # height of image (in 2 bytes) is at 164th position
       img_file.seek(163)

       # read the 2 bytes
       a = img_file.read(2)

       # calculate height
       height = (a[0] << 8) + a[1]

       # next 2 bytes is width
       a = img_file.read(2)

       # calculate width
       width = (a[0] << 8) + a[1]

   print("The resolution of the image is",width,"x",height)

jpeg_res("img1.jpg")

In [ ]:
## Create a stop watch using python.
# importing the time module 
import time

print("Press ENTER to start the stopwatch")
print("and, press CTRL + C to stop the stopwatch")

# infinite loop
while True:
    try:
        input() #For ENTER
        start_time = time.time()
        print("Stopwatch started...")
        
    except KeyboardInterrupt:
        print("Stopwatch stopped...")
        end_time = time.time()
        print("The total time:", round(end_time - start_time, 2),"seconds")
        break # breaking the loop

In [ ]:
## Write a python function to find the standard deviation.
def stdv(X):
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot = tot + (x - mean)**2
    return (tot/len(X))**0.5

# main code
#  a simple data-set 
sample = [1, 2, 3, 4, 5] 
print("Standard Deviation of the sample is: ", stdv(sample))

sample = [1, 2, 3, -4, -5] 
print("Standard Deviation of the sample is: ", stdv(sample))

sample = [10, -20, 30, -40, 50] 
print("Standard Deviation of the sample is: ", stdv(sample))

Standard Deviation of the sample is:  1.4142135623730951
Standard Deviation of the sample is:  3.2619012860600183
Standard Deviation of the sample is:  32.61901286060018


In [ ]:
## Write a function to find the variance.
def variance(X):
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot = tot + (x - mean)**2
    return tot/len(X)

# main code
#  a simple data-set 
sample = [1, 2, 3, 4, 5] 
print("variance of the sample is: ", variance(sample))

sample = [1, 2, 3, -4, -5] 
print("variance of the sample is: ", variance(sample))

sample = [10, -20, 30, -40, 50] 
print("variance of the sample is: ", variance(sample))

variance of the sample is:  2.0
variance of the sample is:  10.64
variance of the sample is:  1064.0


In [ ]:
## Write a code to find the BMI given weight and height.
# getting input from the user and assigning it to user

height = float(2)
weight = float(60)

# the formula for calculating bmi

bmi = weight/(height**2) 
# ** is the power of operator i.e height*height in this case

print("Your BMI is: {0} and you are: ".format(bmi), end='')

#conditions
if ( bmi < 16):
   print("severely underweight")

elif ( bmi >= 16 and bmi < 18.5):
   print("underweight")

elif ( bmi >= 18.5 and bmi < 25):
   print("Healthy")

elif ( bmi >= 25 and bmi < 30):
   print("overweight")

elif ( bmi >=30):
   print("severely overweight")

Your BMI is: 15.0 and you are: severely underweight


In [ ]:
## Write a code to get the total bits in a number.
# count total number of bits in a number 

# declare a number
num = 61

# use bin () method to get the binary value of a number 
# print binary value output will be 0b111101
print ("binary value of {0} is: {1}".format (num, bin(num)))

# store the length of the binary number
length = len(bin(num))

# since binary value contains 0b as prefix
# so to get exact length total number of bits
# subtract 2 from the length
length -=2

# print length
print ("total number of bits: ", length)

binary value of 61 is: 0b111101
total number of bits:  6


In [ ]:
## find the palindrome number goven a list.
# Give size of list
n='12,13,121,666,7676,98'

# Give list of numbers having size n
l=list(map(int,n.strip().split(',')))

print("Palindrome numbers are:")
# check through the list to check 
# number is palindrome or not
for i in l:
    num=str(i)
    if("".join(reversed(num))==num):
        print(i)

Palindrome numbers are:
121
666


In [ ]:
##Write a code  to find the sum of all numbers below 1000 which are multiples of 3 or 5 in the Python programming language?
# initialize the value of n
n=1000 
# initialize value of s is zero.
s=0 

# checking the number is divisible by 3 or 5
# and find their sum
for k in range(1,n+1):
    if k%3==0 or k%5==0: #checking condition 
        s+=k

# printing the result
print('The sum of the number:',s)

The sum of the number: 234168


In [ ]:
##write a code to  remove hyper link from text.
import re
tweet='https://google.com this is the best '
tweet = re.sub(r'https?:\/\/.\S+', "", tweet) 
print(tweet)

 this is the best 


In [ ]:
## Write a code to remove hashtags.
tweet = re.sub(r'#', '', tweet) 
print(tweet)

 this is the best 


In [ ]:
##write a code to remove old style retweet text "RT" 
tweet = re.sub(r'^RT[\s]+', '', tweet) 
print(tweet)

 this is the best 


In [ ]:
## write a code to replace the slangs with actual value.
#dictionary consisting of the contraction and the actual value 
Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will", 
           "'d":" would","'ve":" have","'re":" are"} 
  
#replace the contractions 
for key,value in Apos_dict.items(): 
    if key in tweet: 
        tweet=tweet.replace(key,value) 
  
print("After Contraction replacement the tweet is:-\n{}".format(tweet))

After Contraction replacement the tweet is:-
 this is the best 


In [ ]:
## Wite a code to split the attached words 
import re 
#separate the words 
tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s]) 
print("After spliting attached words the tweet is:-\n{}".format(tweet))


After spliting attached words the tweet is:-
 this is the best 


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
## write a code to remove punctuations.
#for string operations 
import string           
clean_tweet=[] 
tweet_list=['here','comes' ,'the' ,'#', 'rain', ',' ,'agin' ,'fallin' ,'from' ,'sky' ,'!']
#remove punctuations 
for word in tweet_list: 
    #print(word)
    if word not in string.punctuation: 
        clean_tweet.append(word) 
  
print("clean_tweet = {}".format(clean_tweet))

clean_tweet = ['here', 'comes', 'the', 'rain', 'agin', 'fallin', 'from', 'sky']


In [ ]:
## Wite a code to have 3 numbers are print all the possible combinations.
a=99
b=5
c=121
d=[]
d.append(a)
d.append(b)
d.append(c)
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,3):
            if(i!=j&j!=k&k!=i):
                print(d[i],d[j],d[k])

99 5 121
99 121 5
5 99 121
5 121 99
121 99 5
121 5 99


In [ ]:
## Wite a program to print identity matrix.
n=5
for i in range(0,n):
    for j in range(0,n):
        if(i==j):
            print("1",sep=" ",end=" ")
        else:
            print("0",sep=" ",end=" ")
    print()

1 0 0 0 0 
0 1 0 0 0 
0 0 1 0 0 
0 0 0 1 0 
0 0 0 0 1 


In [ ]:
##write a code to see if the number is an armstrong number or not.
n=1
a=list(map(int,str(n)))
b=list(map(lambda x:x**3,a))
if(sum(b)==n):
    print("The number is an armstrong number. ")
else:
    print("The number isn't an arsmtrong number. ")

The number is an armstrong number. 


In [ ]:
## wite a code to compute a polynomial equation given that the coefficients of the polynomial are stored in a list.
import math
#print("Enter the coefficients of the form ax^3 + bx^2 + cx + d")
lst=[]
for i in range(0,4):
    a=5
    lst.append(a)
x=3
sum1=0
j=3
for i in range(0,3):
    while(j>0):
        sum1=sum1+(lst[i]*math.pow(x,j))
        break
    j=j-1
sum1=sum1+lst[3]
print("The value of the polynomial is:",sum1)

The value of the polynomial is: 200.0


In [ ]:
## Wite a function to print the sum of sine series, run the function with user inputs.
import math
def sin(x,n):
    sine = 0
    for i in range(n):
        sign = (-1)**i
        pi=22/7
        y=x*(pi/180)
        sine = sine + ((y**(2.0*i+1))/math.factorial(2*i+1))*sign
    return sine
x=int(input("Enter the value of x in degrees:"))
n=int(input("Enter the number of terms:"))
print(round(sin(x,n),2))

Enter the value of x in degrees:3
Enter the number of terms:4
0.05


In [ ]:
## wite a code to get the two inputs and print the larger one
string1=input("Enter first string:")
string2=input("Enter second string:")
count1=0
count2=0
for i in string1:
      count1=count1+1
for j in string2:
      count2=count2+1
if(count1<count2):
      print("Larger string is:")
      print(string2)
elif(count1==count2):
      print("Both strings are equal.")
else:
      print("Larger string is:")
      print(string1)

Enter first string:A
Enter second string:bbb
Larger string is:
bbb


In [ ]:
## write a code that tprogram takes a string and creates a dictionary with key as first character and value as words starting with that character.
s="those were the best days of my life"
count = 0
vowels = set("aeiou")
for letter in s:
    if letter in vowels:
        count += 1
print("Count of the vowels is:")
print(count)

Count of the vowels is:
10


In [ ]:
## Write a Python Program to check common letters in the two input strings.
s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)&set(s2))
print("The common letters are:")
for i in a:
    print(i)

Enter first string:this is me 
Enter second string:here i am 
The common letters are:
h
e
m
i
 


In [ ]:
## write a code That  takes two strings and displays which letters are in the first string but not in the second string.
s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)-set(s2))
print("The letters are:")
for i in a:
    print(i)


Enter first string:here i am 
Enter second string:this isme 
The letters are:
r
a


In [ ]:
#This is a Python Program to display which letters are in the two strings but not in both.

s1=input("Enter first string:")
s2=input("Enter second string:")
a=list(set(s1)^set(s2))
print("The letters are:")
for i in a:
    print(i)

Enter first string:Here I am 
Enter second string:This is me
The letters are:
h
I
T
r
s
H
a
i


In [ ]:
## Wite a function to get the total sum of list recursively.
def sum1(lst):
    total = 0
    for element in lst:
        if (type(element) == type([])):
            total = total + sum1(element)
        else:
            total = total + element
    return total
print( "Sum is:",sum1([[1,2],[3,4]]))

Sum is: 10


In [ ]:
## write a function to get the length of the list recursively.
def length(lst):
    if not lst:
        return 0
    return 1 + length(lst[1::2]) + length(lst[2::2])
a=[1,2,3]
print("Length of the string is: ")
print(a)

Length of the string is: 
[1, 2, 3]


In [ ]:
#Write a code to read from the file.
a=str(input("Enter the name of the file with .txt extension:"))
file2=open(a,'r')
line=file2.readline()
while(line!=""):
    print(line)
    line=file2.readline()
file2.close()